In [ ]:
import pandas as pd
import requests
import urllib3
import csv
import time
import re
import numpy as np
from random import randint
from pandas import json_normalize
import os

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
path = os.getcwd()
os.chdir(path)

In [ ]:
def get_json(url):
    r = requests.get(url)
    time.sleep(randint(1,3))
    result = json_normalize(r.json())
    return (result)

In [ ]:
records = pd.read_excel('HUD_records_for_geocodio.xlsx')

Using census geocoder to get census tracts of addresses

samples url
https://geocoding.geo.census.gov/geocoder/geographies/address?street=100%20KINGSBURY%20LANE&city=GLASTONBURY&state=CT&zip=6033&benchmark=4&vintage=4&format=json

In [ ]:
records.head()

In [ ]:
for col in records.columns:
    records[col] = records[col].str.strip()

In [ ]:
records['url'] = ['https://geocoding.geo.census.gov/geocoder/geographies/address?street='+ x + '&city=' + y + '&state=' + z + '&zip=' + a + '&benchmark=4&vintage=4&format=json' for (x,y,z,a) in zip(records.Address, records.City, records.State, records.zip5)]

In [ ]:
censusdf = pd.DataFrame()

In [ ]:
for url in list(records.url):
    try:
        js = get_json(url)
        df = pd.DataFrame(js['result.addressMatches'].iloc[0][0]['geographies']['Census Tracts'])
        df['url'] = url
        censusdf = censusdf.append(df)
        print (list(records.url).index(url))
    except Exception:
        standin = {'url': url}
        df = pd.DataFrame(standin, index = [0])
        censusdf = censusdf.append(df)
        print (list(records.url).index(url))
        print ('error')

In [ ]:
censusdf1.to_csv('20210120_census_tract_extract.csv')